# Figures from Water Column Model Runs

In [165]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = 'all'

# Data Science and Visualization Imports
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import xarray as xr
import numpy as np

### Data Imports

In [3]:
wc_results_path = 'results_data/wc_runs_all.nc'
baseline_path = 'results_data/baseline_wc_full.nc'
input_data_path = 'results_data/usgs_data_full.csv'
wc_ds = xr.open_dataset(wc_results_path)
base_wc_ds = xr.open_dataset(baseline_path)
df = pd.read_csv(input_data_path)

In [4]:
base_wc_ds

<xarray.Dataset>
Dimensions:     (Model Runs: 114, index: 80)
Coordinates:
  * index       (index) int32 0 1 2 3 4 5 6 7 8 9 ... 71 72 73 74 75 76 77 78 79
Dimensions without coordinates: Model Runs
Data variables:
    U Velocity  (Model Runs, index) float64 ...
    Q           (Model Runs, index) float64 ...
    Q2          (Model Runs, index) float64 ...
    Q2L         (Model Runs, index) float64 ...
    Z           (Model Runs, index) float64 ...
    Kq          (Model Runs, index) float64 ...
    nu_t        (Model Runs, index) float64 ...
Attributes:
    description:  Model run results for Bay-side sites.

In [5]:
wc_ds

<xarray.Dataset>
Dimensions:     (Model Runs: 114, index: 80)
Coordinates:
  * index       (index) int32 0 1 2 3 4 5 6 7 8 9 ... 71 72 73 74 75 76 77 78 79
Dimensions without coordinates: Model Runs
Data variables:
    U Velocity  (Model Runs, index) float64 ...
    Q           (Model Runs, index) float64 ...
    Q2          (Model Runs, index) float64 ...
    Q2L         (Model Runs, index) float64 ...
    Z           (Model Runs, index) float64 ...
    Kq          (Model Runs, index) float64 ...
    nu_t        (Model Runs, index) float64 ...
Attributes:
    description:  Model run results for Bay-side sites.

Compiling Data from modeling runs for analysis

In [148]:
df1 = pd.DataFrame({
    'mean_u': ([vel.values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_25': ([vel[:20].values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_50': ([vel[:40].values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_75': ([vel[:60].values.mean() for vel in wc_ds['U Velocity']]),
    'Q2': ([val.values.max() for val in wc_ds['Q2']]),
    'Hveg/WC': df['ave_ht']/df['wc_height'],
    'wc_height': df['wc_height'],
    'avg_ht': df['ave_ht'],
    'density_total': df['density_final'],
    'frontal_total': df['frontal_total'],
    'site': df['site'],
    'run': np.repeat('Vegetated Run',a)
})
df2 = pd.DataFrame({
    'mean_u': ([vel.values.mean() for vel in base_wc_ds['U Velocity']]),
    'mean_u_25': ([vel[:20].values.mean() for vel in base_wc_ds['U Velocity']]),
    'mean_u_50': ([vel[:40].values.mean() for vel in base_wc_ds['U Velocity']]),
    'mean_u_75': ([vel[:60].values.mean() for vel in base_wc_ds['U Velocity']]),
    'Q2': ([val.values.max() for val in base_wc_ds['Q2']]),
    'Hveg/WC': df['ave_ht']/df['wc_height'],
    'wc_height': df['wc_height'],
    'avg_ht': df['ave_ht'],
    'density_total': df['density_final'],
    'frontal_total': df['frontal_total'],
    'site': df['site'],
    'run': np.repeat('Baseline Run',a)
})
dff = pd.concat([df1, df2])
dff.sample(5)

,mean_u,mean_u_25,mean_u_50,mean_u_75,Q2,Hveg/WC,wc_height,avg_ht,density_total,frontal_total,site,run
46,1.084220,0.219663,0.601452,0.875571,0.205699,11.674450,0.5996,7.0,5989.570395,11289.142280,Interior,Vegetated Run
113,13.079509,11.807010,12.440616,12.814301,1.008872,16.655101,0.7205,12.0,5359.089301,17315.676881,Interior,Baseline Run
41,11.754988,10.610842,11.180534,11.516528,0.814673,24.034335,0.5825,14.0,4413.367659,16636.630728,Interior,Baseline Run
45,14.304230,12.913005,13.605730,14.014279,1.206870,12.772875,0.8612,11.0,5043.848753,14939.015348,Interior,Baseline Run
38,0.497071,0.003167,0.145223,0.332292,0.161625,26.909852,0.6689,18.0,5359.089301,25973.515321,Creek,Vegetated Run


## EDA and Scatterplots

In [195]:
fig = px.scatter(dff, x='Q2', y='avg_ht', color='site', symbol='run', trendline='ols', template='plotly_dark')
fig.update_layout(
    autosize=True,)
fig.show()

In [196]:
fig = px.scatter(dff, x='mean_u', y='wc_height', trendline='ols', symbol='run', color='site', template='plotly_dark')
fig.update_layout(
    autosize=True,)
fig.show()

In [199]:
fig = px.scatter(dff, x='mean_u_25', y='density_total', symbol='run', color='site', trendline='ols', template='plotly_dark')
#fig = px.scatter(base_df, x='mean_u_25', y='density_total', symbol='site', color='site')
fig.update_layout(
    autosize=True,)
fig.show()

In [200]:
fig = px.scatter(dff, x='mean_u', y='Hveg/WC', symbol='run', color='site', trendline='ols', template='plotly_dark')
#fig = px.scatter(base_df, x='mean_u', y='Hveg/WC', symbol='site', color='site')
fig.update_layout(
    autosize=True,)
fig.show()

## Contour Plots

In [194]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Baseline Runs', 'Vegetated Runs'))

cbarlocs = [.85, .5, .15]

fig.add_trace(go.Contour(
        z=df2['mean_u_25'],
        x=df2['density_total'], # horizontal axis
        y=df2['Hveg/WC'],# vertical axis,
        colorbar_x=0.46,
        line_smoothing=0.85), 1, 1)
fig.add_trace(go.Contour(
        z=df1['mean_u_25'],
        x=df1['density_total'], # horizontal axis
        y=df1['Hveg/WC'],# vertical axis,
        line_smoothing=0.85), 1, 2)

#fig = go.Figure(data =
 #   go.Contour(
  #      z=dff['mean_u_25'],
   #     x=dff['density_total'], # horizontal axis
    #    y=dff['Hveg/WC'],# vertical axis,
     #))
fig.update_layout(
    title="Velocity Contour Plot",
    xaxis_title="Density",
    yaxis_title="Hveg/WC",
    autosize=True,
    template='plotly_dark')
fig.show()

## Maps

## Animations